In [ ]:
import os, glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import segmentation_models_pytorch as smp

# =========================
# Reproducibility
# =========================
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

# =========================
# Dataset
# =========================
class BUSIDataset(Dataset):
    def __init__(self, df, resize=(256, 256)):
        self.df = df.reset_index(drop=True)
        self.resize = resize

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image"]
        mask_path = self.df.iloc[idx]["mask"]

        # Load
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Resize
        if self.resize is not None:
            img = img.resize(self.resize)
            mask = mask.resize(self.resize)

        # To numpy
        img = np.array(img, dtype=np.float32) / 255.0
        mask = (np.array(mask, dtype=np.float32) / 255.0)
        mask = (mask > 0.5).astype(np.float32)

        # To tensor (C,H,W)
        img = torch.from_numpy(img).permute(2, 0, 1)          # (3,H,W)
        mask = torch.from_numpy(mask).unsqueeze(0)            # (1,H,W)
        return img, mask


# =========================
# DATASET PATHS (EDIT THIS)
# =========================
IMG_DIR  = "path/to/BUS-BRA/Images"
MASK_DIR = "path/to/BUS-BRA/Masks"


images = sorted(glob.glob(os.path.join(img_dir, "*.png")))
masks  = sorted(glob.glob(os.path.join(mask_dir, "*.png")))

assert len(images) == len(masks) and len(images) > 0, "Images/Masks count mismatch or empty!"

df = pd.DataFrame({"image": images, "mask": masks})
print(f"Total: {len(df)} image-mask pairs")

# =========================
# Train/Val/Test = 70/15/15
# =========================
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42, shuffle=True)
val_df,   test_df = train_test_split(temp_df, test_size=0.50, random_state=42, shuffle=True)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

train_dataset = BUSIDataset(train_df)
val_dataset   = BUSIDataset(val_df)
test_dataset  = BUSIDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

# =========================
# Loss function: Weighted BCE + Dice
# =========================
def weighted_bce_dice_loss(pred, target):
    t = target.view(-1)
    num_pos = torch.clamp(t.sum(), min=0.)
    num_tot = t.numel()
    num_neg = num_tot - num_pos
    if num_pos <= 0:
        pos_weight = torch.tensor(1.0, device=target.device)
    else:
        pos_weight = (num_neg / (num_pos + 1e-6)).to(target.device)

    bce = F.binary_cross_entropy_with_logits(pred, target, pos_weight=pos_weight)

    p = torch.sigmoid(pred)
    intersection = (p * target).sum(dim=(1,2,3))
    denom = p.sum(dim=(1,2,3)) + target.sum(dim=(1,2,3)) + 1e-6
    dice = (2.0 * intersection + 1e-6) / denom
    dice_loss = 1.0 - dice.mean()

    return bce + dice_loss

@torch.no_grad()
def compute_metrics_from_logits(pred_logits, masks):
    probs = torch.sigmoid(pred_logits)
    preds = (probs > 0.5).float()

    TP = (preds * masks).sum().item()
    FP = (preds * (1 - masks)).sum().item()
    FN = ((1 - preds) * masks).sum().item()
    TN = ((1 - preds) * (1 - masks)).sum().item()

    eps = 1e-7
    precision = TP / (TP + FP + eps)
    recall    = TP / (TP + FN + eps)
    f1        = 2 * precision * recall / (precision + recall + eps)
    iou       = TP / (TP + FP + FN + eps)
    dice      = (2 * TP) / (2 * TP + FP + FN + eps)
    accuracy  = (TP + TN) / (TP + TN + FP + FN + eps)
    return precision, recall, f1, iou, dice, accuracy

# =========================
# Training / Eval loops
# =========================
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    loss_sum = 0.0
    for imgs, masks in loader:
        imgs   = imgs.to(device, non_blocking=True)
        masks  = masks.to(device, non_blocking=True).float()

        optimizer.zero_grad(set_to_none=True)
        logits = model(imgs)
        loss = weighted_bce_dice_loss(logits, masks)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    return loss_sum / max(1, len(loader))

@torch.no_grad()
def eval_loss(model, loader, device):
    model.eval()
    loss_sum = 0.0
    for imgs, masks in loader:
        imgs   = imgs.to(device, non_blocking=True)
        masks  = masks.to(device, non_blocking=True).float()
        logits = model(imgs)
        loss = weighted_bce_dice_loss(logits, masks)
        loss_sum += loss.item()
    return loss_sum / max(1, len(loader))

@torch.no_grad()
def evaluate_metrics(model, loader, device):
    model.eval()
    totals = {"precision":0.0, "recall":0.0, "f1":0.0, "iou":0.0, "dice":0.0, "accuracy":0.0}
    n = 0
    for imgs, masks in loader:
        imgs   = imgs.to(device, non_blocking=True)
        masks  = masks.to(device, non_blocking=True).float()
        logits = model(imgs)
        p, r, f1, iou, d, acc = compute_metrics_from_logits(logits, masks)
        totals["precision"] += p
        totals["recall"]    += r
        totals["f1"]        += f1
        totals["iou"]       += iou
        totals["dice"]      += d
        totals["accuracy"]  += acc
        n += 1
    for k in totals:
        totals[k] /= max(1, n)
    return totals

def run_training(encoder_name, train_loader, val_loader, test_loader, device, epochs=20, lr=1e-4):
    model = smp.Unet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
        activation=None
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    print(f"\n=== Training {encoder_name.upper()} for {epochs} epochs ===")
    for epoch in range(1, epochs+1):
        tr_loss = train_one_epoch(model, train_loader, optimizer, device)
        va_loss = eval_loss(model, val_loader, device)
        print(f"Epoch {epoch:02d}/{epochs} | Train Loss: {tr_loss:.4f} | Val Loss: {va_loss:.4f}")

    test_metrics = evaluate_metrics(model, test_loader, device)
    return model, test_metrics

# =========================
# Run models
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"

encoders_to_test = [
    "resnet34", 
    "resnet50",
    "densenet121",
    "efficientnet-b0",
    "efficientnet-b1",
    "efficientnet-b2",
    "efficientnet-b3",
    "efficientnet-b4",
    "efficientnet-b5",
    "efficientnet-b6",
    "efficientnet-b7",
    "vgg19_bn",
    "mobilenet_v2",   
    "inceptionresnetv2","inceptionv4"
    "mit_b0","mit_b1","mit_b2","mit_b3","mit_b4","mit_b5"          
]
    
results = []
for enc in encoders_to_test:
    print(f"\n🚀 Training with encoder: {enc}")
    _, test_metrics = run_training(
        enc, train_loader, val_loader, test_loader, device,
        epochs=20,
        lr=1e-4
    )
    results.append({
        "Model": enc,
        "Precision": test_metrics["precision"],
        "Recall":    test_metrics["recall"],
        "F1":        test_metrics["f1"],
        "IoU":       test_metrics["iou"],
        "Dice":      test_metrics["dice"],
        "Accuracy":  test_metrics["accuracy"]
    })

df_results = pd.DataFrame(results)
print("\n📊 Final Comparison Results:")
print(df_results)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
Total: 1875 image-m

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]


=== Training RESNET34 for 20 epochs ===
Epoch 01/20 | Train Loss: 1.3284 | Val Loss: 1.0450
Epoch 02/20 | Train Loss: 0.9049 | Val Loss: 0.7865
Epoch 03/20 | Train Loss: 0.6251 | Val Loss: 0.6311
Epoch 04/20 | Train Loss: 0.4697 | Val Loss: 0.4226
Epoch 05/20 | Train Loss: 0.3639 | Val Loss: 0.4319
Epoch 06/20 | Train Loss: 0.2972 | Val Loss: 0.3517
Epoch 07/20 | Train Loss: 0.2521 | Val Loss: 0.3731
Epoch 08/20 | Train Loss: 0.2274 | Val Loss: 0.3596
Epoch 09/20 | Train Loss: 0.1958 | Val Loss: 0.3229
Epoch 10/20 | Train Loss: 0.1769 | Val Loss: 0.3051
Epoch 11/20 | Train Loss: 0.1773 | Val Loss: 0.3455
Epoch 12/20 | Train Loss: 0.1582 | Val Loss: 0.3190
Epoch 13/20 | Train Loss: 0.1438 | Val Loss: 0.3324
Epoch 14/20 | Train Loss: 0.1396 | Val Loss: 0.3422
Epoch 15/20 | Train Loss: 0.1437 | Val Loss: 0.4510
Epoch 16/20 | Train Loss: 0.1733 | Val Loss: 0.6629
Epoch 17/20 | Train Loss: 0.1759 | Val Loss: 0.3269
Epoch 18/20 | Train Loss: 0.1368 | Val Loss: 0.3875
Epoch 19/20 | Train Los

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]


=== Training MOBILENET_V2 for 20 epochs ===
Epoch 01/20 | Train Loss: 1.3264 | Val Loss: 0.9845
Epoch 02/20 | Train Loss: 0.8244 | Val Loss: 0.7058
Epoch 03/20 | Train Loss: 0.5862 | Val Loss: 0.5801
Epoch 04/20 | Train Loss: 0.4436 | Val Loss: 0.4694
Epoch 05/20 | Train Loss: 0.3561 | Val Loss: 0.3978
Epoch 06/20 | Train Loss: 0.2993 | Val Loss: 0.3939
Epoch 07/20 | Train Loss: 0.2624 | Val Loss: 0.3383
Epoch 08/20 | Train Loss: 0.2346 | Val Loss: 0.3589
Epoch 09/20 | Train Loss: 0.2355 | Val Loss: 0.3423
Epoch 10/20 | Train Loss: 0.2250 | Val Loss: 0.3539
Epoch 11/20 | Train Loss: 0.1950 | Val Loss: 0.3660
Epoch 12/20 | Train Loss: 0.1730 | Val Loss: 0.3420
Epoch 13/20 | Train Loss: 0.1624 | Val Loss: 0.3781
Epoch 14/20 | Train Loss: 0.1517 | Val Loss: 0.3530
Epoch 15/20 | Train Loss: 0.1450 | Val Loss: 0.4044
Epoch 16/20 | Train Loss: 0.1784 | Val Loss: 0.3624
Epoch 17/20 | Train Loss: 0.1627 | Val Loss: 0.3135
Epoch 18/20 | Train Loss: 0.1587 | Val Loss: 0.4074
Epoch 19/20 | Train

config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.3M [00:00<?, ?B/s]


=== Training MIT_B0 for 20 epochs ===
Epoch 01/20 | Train Loss: 1.4211 | Val Loss: 1.0316
Epoch 02/20 | Train Loss: 0.9487 | Val Loss: 0.7676
Epoch 03/20 | Train Loss: 0.7221 | Val Loss: 0.6233
Epoch 04/20 | Train Loss: 0.5694 | Val Loss: 0.4973
Epoch 05/20 | Train Loss: 0.4499 | Val Loss: 0.4251
Epoch 06/20 | Train Loss: 0.3595 | Val Loss: 0.3756
Epoch 07/20 | Train Loss: 0.3294 | Val Loss: 0.3645
Epoch 08/20 | Train Loss: 0.2751 | Val Loss: 0.3387
Epoch 09/20 | Train Loss: 0.2556 | Val Loss: 0.3125
Epoch 10/20 | Train Loss: 0.2386 | Val Loss: 0.3187
Epoch 11/20 | Train Loss: 0.2053 | Val Loss: 0.3194
Epoch 12/20 | Train Loss: 0.2173 | Val Loss: 0.3261
Epoch 13/20 | Train Loss: 0.2292 | Val Loss: 0.3272
Epoch 14/20 | Train Loss: 0.1880 | Val Loss: 0.3999
Epoch 15/20 | Train Loss: 0.1717 | Val Loss: 0.4327
Epoch 16/20 | Train Loss: 0.1614 | Val Loss: 0.3729
Epoch 17/20 | Train Loss: 0.1579 | Val Loss: 0.3193
Epoch 18/20 | Train Loss: 0.1576 | Val Loss: 0.3309
Epoch 19/20 | Train Loss: